# Example ticket generation workflow

This notebook sets up an environment, queries mediacloud for sources, creates Source objects and runs 'issues' on them, then generates tickets in zammad out of the detected issues

In [1]:
#%pip install -q -e  ../. 
import os

mc_token = "BYO"
os.environ["MC_API_TOKEN"] = mc_token
os.environ["ZAMMAD_TOKEN"] = "BYO"

from directory_issues import Collection, Source
from directory_issues.clients.zammad_client import ZammadClient
from IPython.display import HTML

## Use the mediacloud directory to get a specific subset of sources
In this case, those that have a defined 'url search string'


In [2]:
import mediacloud
mc_dir = mediacloud.api.DirectoryApi(mc_token)
sources_with_url_search_strings = []

more_sources = True
offset = 0
limit=100000
while more_sources:
    
    resp = mc_dir.source_list(limit = limit, offset=offset)
    matches = [r for r in resp["results"] if r["url_search_string"] is not None]
    S = [Source(r) for r in matches]
    for s in S:
        print(s)
        s.find_issues()
    sources_with_url_search_strings.extend(S)
    print(len(sources_with_url_search_strings))
    if resp["next"] is not None:
        offset += limit
    else:
        more_sources = False
        

Source(indiatimes.com, http://timesofindia.indiatimes.com/)
Source(ctvnews.ca, http://bc.ctvnews.ca/)
Source(nation.africa, https://nation.africa/)
Source(theguardian.com, https://www.theguardian.com/us)
Source(intoday.in, http://indiatoday.intoday.in/#spider)
Source(cbslocal.com, https://www.cbsnews.com/boston/)
Source(forward.com, http://forward.com)
Source(bizjournals.com, http://www.bizjournals.com/boston)
Source(uol.com.br, http://www.folha.uol.com.br/)
Source(iol.co.za, http://www.iol.co.za/the-star)
Source(cbslocal.com, http://sanfrancisco.cbslocal.com)
Source(bizjournals.com, http://www.bizjournals.com/sanfrancisco/)
Source(bdlive.co.za, http://www.bdlive.co.za/)
Source(kenyanews.go.ke, https://www.kenyanews.go.ke/)
Source(ctvnews.ca, http://regina.ctvnews.ca/)
Source(ctvnews.ca, http://windsor.ctvnews.ca/)
Source(news12.com, http://connecticut.news12.com/)
Source(nation.africa, https://nairobinews.nation.africa/)
Source(cbslocal.com, http://newyork.cbslocal.com)
Source(india.c

## A demo source ticket with these issues


In [3]:
sources_with_issues = [s for s in sources_with_url_search_strings if len(s.source_issues) != 0]
source = sources_with_issues[10]
print(source.source_data.id)
print(source.source_data.label)
message = source.render_template()
HTML(message)


1912220
btv.co.ke


## Using the zamad client to create tickets 
the source class has a method wrapped inside for this task. 

In [4]:
len(sources_with_issues)

16

In [5]:
for source in sources_with_issues:
    source.post_zammad_issue()

HTTPError: {"error":"Collections is longer than the allowed length 120","error_human":"Collections is longer than the allowed length 120"}

In [ ]:
FIXED_USS_LIST = []
issue_map = {source:[i["issue_name"] for i in source.source_issues] for source in sources_with_issues}
import pprint
pprint.pprint(issue_map)

In [ ]:
postfixes = [s for s,i in issue_map.items() if 'USS-*-Postfix' in i and len(i) == 1]

In [ ]:
for s in postfixes:
    print(s)
    print(s.source_data.url_search_string)
    fixed_uss = s.source_data.url_search_string + "/*"
    print(fixed_uss)
    update = {"name":s.source_data.name, "id":s.source_data.id, "platform":"online_news", "url_search_string":fixed_uss}
    FIXED_USS_LIST.append(update)

In [ ]:
httpprefixes = [s for s,i in issue_map.items() if 'USS-HTTP-Prefix' in i ]
for s in httpprefixes:
    print(s)
    print(s.source_data.url_search_string)
    fixed_uss = s.source_data.url_search_string.split("//")[1]
    print(fixed_uss)
    update = {"name":s.source_data.name, "id":s.source_data.id, "platform":"online_news", "url_search_string":fixed_uss}
    FIXED_USS_LIST.append(update)

In [ ]:
for s, issue in issue_map.items():
    if s.source_data.id not in [source["id"] for source in FIXED_USS_LIST]:
        if len(issue) == 3:
            print(s)
            print(issue)
            
            fixed_uss = None
            update = {"name":s.source_data.name, "id":s.source_data.id, "platform":"online_news", "url_search_string":fixed_uss}
            FIXED_USS_LIST.append(update)

In [ ]:
pprint.pprint(FIXED_USS_LIST)
import json
json.dumps(FIXED_USS_LIST)

In [ ]:
#monkey-patch!?
## A quick but terrifying attempt at building a method to bulk-edit the source values.
from types import MethodType

def patch_source_url_search_string(self, id, url_search_string):
    endpoint_url = self.BASE_API_URL + f"sources/sources/{id}/"
    og = self.source(id)
    og["url_search_string"] = url_search_string
    print(og)
    og = {k: v for k, v in og.items() if v is not None}
    ref = f"https://search.mediacloud.org/sources/{id}"
    self._session.get(ref)
    csrf_token = self._session.cookies.get("csrftoken", "null")
    self._session.headers.update({"x-csrftoken":csrf_token})
    self._session.headers.update({"Referer":ref})
    self._session.headers.update({"Origin":"https://search.mediacloud.org"}) 
    self._session.cookies.update({"sessionid":"nuh-uh"})

    response = self._session.patch(endpoint_url, json=og)
    print(response.status_code)

mc_dir.patch_source_url_search_string = MethodType(patch_source_url_search_string, mc_dir)

In [ ]:
#print(mc_dir.source(ex.source_data.id)["url_search_string"])
#mc_dir.patch_source_url_search_string(ex.source_data.id, ex.source_data.url_search_string+"/*")
#print(mc_dir.source(ex.source_data.id)["url_search_string"])

In [ ]:
for fix in FIXED_USS_LIST:
    pass
    #print(mc_dir.source(fix["id"])["url_search_string"])
    #mc_dir.patch_source_url_search_string(fix["id"], fix["url_search_string"])
    #print(mc_dir.source(fix["id"])["url_search_string"])

In [ ]:
mc_dir._session.cookies